In [15]:
import whisper
# from pytube import YouTube
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress
import json


def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            print(f"--- Directory created successfully: {directory} ---")
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def youtube_video_download(url,main_directory):
    yt = YouTube(url, on_progress_callback = on_progress)
     
    ys = yt.streams.get_highest_resolution()

    vid_title=yt.title.replace(" ","_")
    output_path=f"{main_directory}/{vid_title}/"
    createDirectory(output_path)
    filename=f"youtube_video.mp4"
    
    file_path = os.path.join(output_path, filename)
    
    if os.path.isfile(file_path):
        print(f"--- Video already exists")
        pass
    else: 
        print(f"--- Download ... {yt.title} ---")
        ys.download(output_path=output_path, filename=filename)
        print(f"--- Video Download Done ---")
    
    return output_path, filename

def stt_with_whisper(directory, video_path):
    print("--- Whisper Model load ---")
    whisper_model = whisper.load_model("turbo")
    # transcription = whisper_model.transcribe(f"./{yt.title}.mp4", fp16 = False)["text"].strip()
    print("--- Start Transcribe ---")
    path=directory+video_path
    stt_result=whisper_model.transcribe(path, fp16 = False)

    return stt_result

def process_stt_by_minute(stt_data):
    print("--- segment stt result ---")
    result = []
    
    segments = stt_data['segments']
    
    if not segments:
        return result
        
    current_minute = 0
    current_texts = []
    current_start = 0
    
    for segment in segments:
        segment_start_minute = int(segment['start'] // 60)
        
        if segment_start_minute == current_minute:
            current_texts.append(segment['text'].strip())
        else:
            if current_texts:
                result.append({
                    "start": current_start,
                    "end": (current_minute + 1) * 60,
                    "text": " ".join(current_texts)
                })
            
            while current_minute + 1 < segment_start_minute:
                current_minute += 1
                result.append({
                    "start": current_minute * 60,
                    "end": (current_minute + 1) * 60,
                    "text": ""
                })
            
            current_minute = segment_start_minute
            current_texts = [segment['text'].strip()]
            current_start = current_minute * 60
    
    if current_texts:
        result.append({
            "start": current_start,
            "end": (current_minute + 1) * 60,
            "text": " ".join(current_texts)
        })
    
    return result

def save_captions_to_file(directory,result):
    stt_output_path=f"{directory}/transcription.json"
    with open(stt_output_path, 'w', encoding='utf-8') as outfile:
        json.dump(result, outfile,indent="\t",ensure_ascii=False)

In [16]:
url="https://youtu.be/95vT5rBiEYI?si=ruI2oOOIQLib8pDj" 
main_directory="/workspace/.gen/VLM/Video_to_chat/assets"
directory, video_path=youtube_video_download(url,main_directory)
stt_result=stt_with_whisper(directory, video_path)
minute_res=process_stt_by_minute(stt_result)
save_captions_to_file(directory,minute_res)

--- Directory created successfully: /workspace/.gen/VLM/Video_to_chat/assets/[#유퀴즈온더블럭]_💥지드래곤의_컴백을_격하게_환영합니다💥_7년_만에_신곡_〈POWER〉로_돌아온_지디💕/ ---
--- Download ... [#유퀴즈온더블럭] 💥지드래곤의 컴백을 격하게 환영합니다💥 7년 만에 신곡 〈POWER〉로 돌아온 지디💕 ---
--- Video Download Done ---█████████████████████| 100.0%
--- Whisper Model load ---
--- Start Transcribe ---
--- segment stt result ---


In [3]:
minute_res=process_stt_by_minute(stt_result)

In [5]:
len(minute_res)

17

In [6]:
minute_res

[{'start': 0,
  'end': 60,
  'text': '아이고! 아이고! 아이고! 아이고! 야, 이 드랙랑! 예! 야, 이거만이야! 아, 참! 아... 아... 왜, 왜? 오랜만이라... 어서오세요! 아우! 아우, GD! 나... 박수라도 한 번! 괜찮아요? 네, 아이... 나 반가울 거야! 많이 반가울 거야! 네... 네, 이제... G-Dragon! 아, 아! 아, 우리 또 GD 진짜 오랜만이네요! GD는 첫 출연이니까!'},
 {'start': 60,
  'end': 120,
  'text': '그렇죠! 그죠, 그죠? 네! 네... 진짜로... 제가... 이런... 토크? 해야라고 해야 되나요? 네, 네! 한 10년 만 정도라서... 맞아, 맞아! 지금 얘기 들어보니까 거의 힐링캠프가 마지막이었다고! 네... 네, 그죠? 오늘도 무슨 캠프로 온 것 같긴 한데... 그렇죠? 네... 네... 자, 제가 소활드릴게요! 영원한 건 절대 없어를 외쳤지만 영원한 시대 아이콘 우리 GDragon! 네! 권영수입니다! 팬들에게 juvenile 기억나는 기 limb 특이였 bipolar duct극! enables�bury, 그냥 DisseCO wozus는 롤비'},
 {'start': 120,
  'end': 180,
  'text': '영원한건 절대 없어 결국엔 너 내게 있지 있어 사랑같은 소리 따위지 오늘 밤은 빛다가 세상을 헌는다 세상을 헌는다 오랜만에 인사를 좀 해주세요 네 안녕하세요. 이거 제가 왜 하겠... 네 안녕하세요. 가수 지드래골입니다. 네 반갑습니다. 올해가 이제 값진 연이죠? 올해가 얼마 안 남긴 했습니다만 푸른 용의 해입니다. 지옹의 해를 맞아서 드디어 7년 만에! 이야 이게 7년 만이네. 솔로 앨범이! 진짜 많은 분들이 GD가 또 어떤 곡을 발표할까? 어떤 곡을 또 들고 나올까? 관심도 굉장히 좀 컸었고. 실제로 부담도 좀 될 것 같아. GD 입장에서도. 네. 부담을 가진 지는 너무... 오래됐다고 해야 되나?'},
 {'start